In [ ]:
import collections
import itertools
from octo_spork.expressions import *
from octo_spork.logic import to_cnf, to_dnf
from octo_spork.domain import simplify_intervals, simplify_sets, get_attributes

In [ ]:
x1, x2, x3 = [Attribute('x{}'.format(i+1)) for i in range(3)]
expression = And([
    And([Gt(x1, 1), Lt(x1, 2), Ge(x2, 1)]),
    Or([Not(Ge(x1, 1.5)), Lt(x2, 0)])])
print(nice_repr(expression))

In [ ]:
reduced = to_dnf(expression)
print(nice_repr(reduced))
type(reduced)

In [ ]:
expr = list(reduced.clauses)[0]
nice_repr(expr)
len(expr.clauses)

In [ ]:
def simplify_reduce(expression):
    if isinstance(expression, And):
        by_attribute = collections.defaultdict(list)
        for clause in expression.clauses:
            by_attribute[frozenset(get_attributes(clause))].append(clause)
        return And([
            simplify_intervals(And(clausegroup))
            for clausegroup in by_attribute.values()])
    return expression

In [ ]:
def simplify(expression, depth=0):
    if depth > 10:
        return expression
    if len(get_attributes(expression)) == 1:
        try:
            return simplify_intervals(expression)
        except:
            try:
                return simplify_sets(expression)
            except:
                pass
    if isinstance(expression, And):
        by_attribute = collections.defaultdict(list)
        for clause in expression.clauses:
            by_attribute[frozenset(get_attributes(clause))].append(clause)
        return And([
            simplify(And(clausegroup), depth=depth+1)
            for clausegroup in by_attribute.values()])
    if isinstance(expression, Or):
        by_attribute = collections.defaultdict(list)
        for clause in expression.clauses:
            by_attribute[frozenset(get_attributes(clause))].append(clause)
        return Or([
            simplify(Or(clausegroup), depth=depth+1)
            for clausegroup in by_attribute.values()])
    return expression

In [ ]:
expr = And([Gt(x1, 1), Gt(x1, 2)])
print(nice_repr(expr))
print(nice_repr(simplify(expr)))

In [ ]:
expr = And([In(x1, [1, 2, 3, 4]), In(x1, [3, 4, 5, 6])])
print(nice_repr(expr))
print(nice_repr(simplify(expr)))

In [ ]:
expr = And([Gt(x1, 1), Gt(x1, 2), In(x2, [1, 2, 3, 4]), In(x2, [3, 4, 5, 6])])
print(nice_repr(expr))
print(nice_repr(simplify(expr)))

In [ ]:
expr = Or([Gt(x1, 1), Gt(x1, 2)])
print(nice_repr(expr))
print(nice_repr(simplify(expr)))

In [ ]:
expr = Or([In(x2, [1, 2, 3, 4]), In(x2, [3, 4, 5, 6])])
print(nice_repr(expr))
print(nice_repr(simplify(expr)))

In [ ]:
expr = Or([Gt(x1, 1), Gt(x1, 2), In(x2, [1, 2, 3, 4]), In(x2, [3, 4, 5, 6])])
print(nice_repr(expr))
print(nice_repr(simplify(expr)))

In [ ]:
expr = And([
    And([Gt(x1, 1), Lt(x1, 2), Ge(x2, 1)]),
    Or([Not(Ge(x1, 1.5)), Lt(x2, 0)])])

In [ ]:
# This works! But the general strategy has a recursion problem.

print(expr)
print(type(to_dnf(expr)))
print()

a, b = to_dnf(expr).clauses
print(nice_repr(a))
print(nice_repr(b))
print()

sa = to_cnf(simplify(a))
sb = to_cnf(simplify(b))
print(nice_repr(sa))
print(nice_repr(sb))
print()

result = to_cnf(Or([sa, sb]))
print(nice_repr(result))